# Text Generation with Neural Networks (continued)

### In the previous notebook, a model has been created and trained. Here it is imported and tested.

In [1]:
import nltk
import string
import json
import keras
import tensorflow as tf
import gradio as gr
import numpy as np
import keras.utils

from keras.utils import pad_sequences, to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import load_model, Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from nltk.tokenize import sent_tokenize
from random import randint
from pickle import load
from sklearn.model_selection import train_test_split

2023-04-07 12:37:11.607619: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 12:37:11.799938: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-07 12:37:11.799962: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-07 12:37:12.750161: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

#### Importing the model :

In [2]:
with open("model_amtextgen.json", "r") as json_file:
    loaded_model_json = json_file.read()

loaded_model = tf.keras.models.model_from_json(loaded_model_json)

loaded_model.load_weights("model.h5")

2023-04-07 12:37:15.924484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-07 12:37:15.924513: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-07 12:37:15.924536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (justine-XPS13): /proc/driver/nvidia/version does not exist
2023-04-07 12:37:15.924868: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Importing the variables necessary to the model :

In [4]:
with open("max_len_ref.txt", "r") as file:
    max_len_txt = file.read()
    max_len = int(max_len_txt)

In [5]:
with open("cleaned_ref.txt", "r") as file:
    full = file.read().split('\n')
    cleaned = full[:-1]                        

#### The model can finally be tested in this last part :

"Seed_text" is the text that will be continued by the model, it corresponds to the variable "text" and can be modified as desired. It is also possible to change the number of words to output.

Before generating the text, a tokenisation is done, as well as a padding task. The model is then used.

In [6]:
def generate_text(seed_text):
    next_words = 50
    max_sequence_len = max_len
    corpus = cleaned
    tokenizer = Tokenizer()
    
    tokenizer.fit_on_texts(corpus)
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        
        predict_x = loaded_model.predict(token_list, verbose=0)
        predicted = np.argmax(predict_x,axis=1)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    seed_text += "."
    return seed_text

Generating text using the created function :

In [7]:
texte = "I opened the door and"
generate_text(texte)

'I opened the door and towards the light of sleeping with his middle of the room thinking it never be heard of a room light in a most days and and and there from them did my other while over the alleghanian ridge in the forecastle sign of time a rag unless i will not.'

Or generating text with a nice looking API interface :

In [8]:
demo = gr.Interface(fn=generate_text, inputs="text", outputs="text", title= "American literature text generator")
demo.launch(share=True)

IMPORTANT: You are using gradio version 3.13.0, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c1c1f9a9ad0f8f08.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
